<a href="https://colab.research.google.com/github/YinHuanyu/FPGA/blob/master/Robust_CVD_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [CVPR 2021] Robust Consistent Video Depth Estimation

<img src="https://robust-cvd.github.io/Robust_Consistent_Video_Depth_Estimation_files/teaser.png" width="800">

Welcome to Robust CVD! This is the official colab notebook.

[[Project Page](https://robust-cvd.github.io/)] [[Code](https://github.com/facebookresearch/robust_cvd)]

You can make a copy of this notebook or play with `File -> Open in playground mode` and make changes there.

__DO NOT__ request access to this notebook.

> NOTE: you might need to manually restart the Colab runtime for installing Detectron2 if necessary.

# System Info

## GPU & RAM

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Aug  3 06:07:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


# Prepare the Environment

### Packages from conda (`2-3 mins`)
(might need clicking to restart the runtime if required)

In [ ]:
%%capture
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.7.12-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local

if [ ! -f $MINICONDA_INSTALLER_SCRIPT ]; then
    wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
    chmod +x $MINICONDA_INSTALLER_SCRIPT
    ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
fi

conda install -y -c defaults conda python=3.6
conda update -y -c defaults --all

In [ ]:
!pip install -U torch torchvision cython moviepy psutil scipy tensorboard ipykernel
!pip install 'git+https://github.com/facebookresearch/fvcore'
# NOTE: We need an older version of opencv-contrib to run SURF. We also have to use Python3.6 in results.
!pip install -U opencv-contrib-python==3.4.2.16 opencv-python

# NOTE: We need glog=0.4.0 here due to one introduced incompatibility issue in 0.5.0.
!conda install -y -c conda-forge pybind11 ceres-solver eigen gtest gflags glog=0.4.0 av

     |████████████████████████████████| 831.4 MB 10 kB/s 
     |████████████████████████████████| 22.1 MB 1.5 MB/s 
     |████████████████████████████████| 2.0 MB 14.0 MB/s 
     |████████████████████████████████| 388 kB 50.9 MB/s 
     |████████████████████████████████| 291 kB 50.6 MB/s 
     |████████████████████████████████| 25.9 MB 36 kB/s 
     |████████████████████████████████| 6.0 MB 41.9 MB/s 
     |████████████████████████████████| 120 kB 55.0 MB/s 
     |████████████████████████████████| 14.8 MB 157 kB/s 
     |████████████████████████████████| 3.0 MB 34.3 MB/s 
     |████████████████████████████████| 3.3 MB 32.7 MB/s 
     |████████████████████████████████| 26.9 MB 87 kB/s 
     |████████████████████████████████| 781 kB 52.1 MB/s 
     |████████████████████████████████| 4.3 MB 38.1 MB/s 
     |████████████████████████████████| 4.9 MB 51.9 MB/s 
     |████████████████████████████████| 288 kB 52.7 MB/s 
     |████████████████████████████████| 152 kB 52.1 MB/s 
     |██████████

  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-svaf89qm
  Running command git clone -q https://github.com/facebookresearch/fvcore /tmp/pip-req-build-svaf89qm
     |████████████████████████████████| 640 kB 4.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=64488 sha256=adc8f1e0a855f0ddb879627541d02ca8931adef8f5523ca5dd8346c61bf2686b
  Stored in directory: /tmp/pip-ephem-wheel-cache-skbi04ve/wheels/e5/d4/51/97eb3e4f9feb64f30d12e2d0bc3ec7ce8a9c23a1b9b9add88e
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=d3a02582b818b4cfdbd75fb50fe9cd87e1555288f0d4e0f892988008a11218cc
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
Successfully built fvcore termcolor
     |████████████████████████████████| 30.6 MB 23 kB/s 
     |████████████████████████████████| 49.9 MB 16 kB/s 
Solving environment: - \ | / - \ | / - \ | / 

### Packages from apt-get (`~1min`)

In [ ]:
%%capture
!sudo apt install \
    git \
    cmake \
    vim \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libdouble-conversion-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libatlas-base-dev \
    libfreeimage-dev \
    protobuf-compiler \
    libprotobuf-dev \
    libglew-dev \
    libglvnd-dev \
    libopencv-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev \
    libgtk2.0-dev \

### Install Detectron2 (`2-3 mins`)
(might need clicking to restart the runtime after installation if required)

In [ ]:
%%bash
DETECTRON_REPO=/content/detectron2

if [ ! -d $DETECTRON_REPO ]; then
    git clone https://github.com/facebookresearch/detectron2 detectron2
    pip install -e detectron2
fi

Obtaining file:///content/detectron2
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=ead0faab1e20fc78f87e4dfaf0dd06130a331bf6198460d04e1270544a868f4d
  Stored in directory: /root/.cache/pip/wheels/a8/04/35/9449686f1c26ff16f6224dc942e108329f3782185802ec6b93
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=273365 sha256=b0751dfb065aaf616ad578c1cc772a530b517430630764cf0a6569f9d91b82e5
  Stored in directory: /root/.cache/pip/wheels/d8/c2/ba/8f5306f921c2e79ad7b09effdfed6bd966cfcf8c6fe55422d6
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=1f10d161f1eb9d7930b8b44b9a7a7b810ff6a16128518a8aba8bef4ac59c456b
  Stored in directory: /root/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built antlr4-python3-runtime pycocotools future
  Running setup.py develop for detectron2


Cloning into 'detectron2'...


### Download the robust_cvd codebase and compile (`2-3 mins`)

> Will switch to git cloning from our public repo soon.

In [ ]:
%%bash
sudo apt install libfmt-dev nlohmann-json-dev
cd /content
if [ ! -d /content/cvd2 ]; then
git clone https://github.com/facebookresearch/robust_cvd.git cvd2
fi
mkdir -p /content/cvd2/lib/build && cd /content/cvd2/lib/build
cmake ..
make -j16
export PYTHONPATH="${PYTHONPATH}:/content/cvd2/lib/build"

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  libfmt-doc
The following NEW packages will be installed:
  libfmt-dev nlohmann-json-dev
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 442 kB of archives.
After this operation, 10.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 nlohmann-json-dev all 2.1.1-1.1 [364 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfmt-dev amd64 4.0.0+ds-2 [77.6 kB]
Fetched 442 kB in 2s (218 kB/s)
Selecting previously unselected package nlohmann-json-dev.
(Reading database ... 169065 files and directories currently installed.)
Preparing to unpack .../nlohmann-json-dev_2.1.1-1.1_all.deb ...
Unpacking nlohmann-json-dev (2.1.1-1.1) ...
Selecting previously unselected package libfmt-dev.
Preparing to unpack .../libfmt-dev_4.0.0+ds-2_amd64.deb ...
Unpacking libfmt-dev (4.0.0+ds-2) ...
Setting up libfmt-de



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Cloning into 'cvd2'...


### Download pretrained models (`~1min`)
for estimation of monodepth, optical flow, and dynamic mask

> Try this [Weiyun link](https://share.weiyun.com/ESR1g1hU) if you cannot properly access Dropbox

In [ ]:
%%capture
%%bash
if [ ! -d /content/cvd2/models ]; then
    cd /content/cvd2
    wget https://www.dropbox.com/s/avruiwv95c5xucn/models.zip?dl=1 -O models.zip
    unzip models.zip
fi

# Prepare Input Video

### Try the sample video

Currently the `--frame_range` parameter will only determine the frames to be filtered, but all input frames will still be pre-processed for depth prediction, optical flow estimation, and dynamic mask generation, which would take quite some time for long sequences. So we prepare clips with various lengths for the same sample video to acclerate it here in notebook.

It's recommended to try the short 15-frame sample video first, for a fast initial experience.


In [ ]:
%pushd /content
# 15 frames
!wget https://www.dropbox.com/s/vpglmx6fonaq8eu/family_run_15_frames.mov?dl=1 -O family_run.mov

# 30 frames
# !wget https://www.dropbox.com/s/b6n9ksezeoox9pi/family_run_30_frames.mov?dl=1 -O family_run.mov

# 60 frames
# !wget https://www.dropbox.com/s/x1cpkxy7csxlg0z/family_run_60_frames.mov?dl=1 -O family_run.mov

# 120 frames
#!wget https://www.dropbox.com/s/ds56if4qgcgla65/family_run_120_frames.mov?dl=1 -O family_run.mov

# 600+ frames
#!wget https://www.dropbox.com/s/3lipk0n6crzm697/family_run_600frames_full.mov?dl=1 -O family_run.mov
%popd

/content
--2021-08-03 06:19:03--  https://www.dropbox.com/s/vpglmx6fonaq8eu/family_run_15_frames.mov?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/vpglmx6fonaq8eu/family_run_15_frames.mov [following]
--2021-08-03 06:19:03--  https://www.dropbox.com/s/dl/vpglmx6fonaq8eu/family_run_15_frames.mov
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce3e4f31c5729ca48e5720cdf36.dl.dropboxusercontent.com/cd/0/get/BTighU1O78ztzB7nvvIGG6W9df-3u_uT6s_MCPOjRmM6a4MYX9AVDrMkKyk7L0ZVP3aTD0n4ARPFVxNvM4tqvD7gsQ2-vXJp9LIh44Ao_mNUUl-pVOCEivRWEcQAj3bil_9ubYAKRxt9oZpubXo-socu/file?dl=1# [following]
--2021-08-03 06:19:03--  https://uce3e4f31c5729ca48e5720cdf36.dl.dropboxusercontent.com/cd/0/get/BTighU1O78ztzB7nvvIGG6W9df-3u_uT6s_MCPOj

### or upload your own video if any (uncomment to run)

> Remember to change the input file name for following commands accordingly.

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# Processing

Currently 100 frames might take 15-25mins to process based on the research code on Colab. Without refactoring and optimization it's still relatively slow. Also the precompiled Ceres-solver from conda-forge can only utilize 4 threads at most, which further slows down the whole processing.

> You can directly download the sample output dir from [here](https://drive.google.com/drive/folders/1PoCVH-POuI5qZA8l5OkonCXusq_4ZI3L?usp=sharing) if you just wanted to check the results.

**We plan to accelerate the whole pipeline soon (especially for optical flow computation).**

Please refer to our [supplementary doc](https://robust-cvd.github.io/robust_cvd_supp.pdf) for more analysis.

![](https://robust-cvd.github.io/runtime_analysis.jpg)

```
Args:
    --video_file  : path to your input video
    --path       : path to the output folder which contains all processed results
    --frame_range : frames you want to process for the input video
```


### Main pipeline

> Better only change `--video_file`, `--path`, and `--frame_range` if you don't understand the params.

In [ ]:
%cd /content/cvd2/

!python main.py \
--video_file /content/family_run.mov \
--path /content/family_run_output \
--save_intermediate_depth_streams_freq 1 \
--num_epochs 0 --post_filter \
--opt.adaptive_deformation_cost 10 \
--frame_range 0-10 \
--save_depth_visualization \

/content/cvd2
['/content/cvd2', '/env/python', '/usr/local/lib/python36.zip', '/usr/local/lib/python3.6', '/usr/local/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/site-packages', '/content/detectron2', '/content/cvd2/main.py', 'lib/build']
['/content/cvd2', '/env/python', '/usr/local/lib/python36.zip', '/usr/local/lib/python3.6', '/usr/local/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/site-packages', '/content/detectron2', '/content/cvd2/main.py', 'lib/build', '/content/cvd2/process.py', '/content/cvd2/lib/build']
Using 1 GPUs.
C++ *NOT* logging to Stdout.
------------ Parameters -------------
align: 32
batch_size: 2
display_freq: 100
distance_alpha: 1
distance_scale: 1
distance_type_smooth: l1
distance_type_static: l1
exp_tag: short
filter_radius: 4
flow_model: raft
flow_ops: ['hierarchical2']
frame_range: '0-10'
lambda_contrast_loss: 1.0
lambda_contrast_thresh: 1.05
lambda_disparity_smooth: 0.0
lambda_parameter: 0
lambda_scene_flow_static: 0.0
lambda_smooth_depth_r

# Result Visualization

### Side-by-side video comparison

> You might need to change the hardcoded path based on `frame_range`, such as `R0-10_hierarchical2*`.

In [ ]:
%%capture
%cd /content/cvd2/

import os
import os.path as osp
from utils.visualization import visualize_depth_dir

# Change based on your output path.
output_dir = "family_run_output"

depth_midas_dir = osp.join("/content", output_dir, "depth_midas2/depth")
depth_vis_midas_dir = osp.join("/content", output_dir, "depth_vis_midas2")
os.makedirs(depth_vis_midas_dir, exist_ok=True)
visualize_depth_dir(depth_midas_dir, depth_vis_midas_dir)

depth_result_dir = osp.join("/content", output_dir, "R0-10_hierarchical2_midas2/StD100.0_StR1.0_SmD0_SmR0.0/depth_e0000/e0000_filtered/depth/")
depth_vis_result_dir = osp.join("/content", output_dir,"depth_vis_result")
os.makedirs(depth_vis_result_dir, exist_ok=True)
visualize_depth_dir(depth_result_dir, depth_vis_result_dir)

In [ ]:
%%capture
import glob
import moviepy as mvp
from moviepy.editor import *

fps = 10

color_dir = osp.join("/content", output_dir, "color_down_png")
clip_color = ImageSequenceClip(color_dir, fps=fps)
clip_midas = ImageSequenceClip(depth_vis_midas_dir, fps=fps)
clip_result = ImageSequenceClip(depth_vis_result_dir, fps=fps)

clip_color = clip_color.set_duration(clip_result.duration)
clip_midas = clip_midas.set_duration(clip_result.duration)

clip_color.write_videofile('clip_color.mp4', fps=fps)
clip_midas.write_videofile('clip_midas.mp4', fps=fps)
clip_result.write_videofile('clip_result.mp4', fps=fps)

### Comparison: `original video | MiDaS depth | Our depth`

In [ ]:
video_color = VideoFileClip('clip_color.mp4')
video_midas = VideoFileClip('clip_midas.mp4')
video_result = VideoFileClip('clip_result.mp4')

video = clips_array([[video_color, video_midas, video_result]])
video.write_videofile('video_comparison.mp4', fps=24, codec='mpeg4')

ipython_display(video, autoplay=1, loop=1)

[MoviePy] >>>> Building video video_comparison.mp4
[MoviePy] Writing video video_comparison.mp4


100%|██████████| 27/27 [00:00<00:00, 146.37it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_comparison.mp4 




 92%|█████████▏| 11/12 [00:00<00:00, 66.69it/s]
